# CGAN Implementation

In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

## Generated Image folder

In [2]:
os.makedirs("images", exist_ok=True)
print(">>> Making images directory...")

>>> Making images directory...


## Hyper-parameter setting

In [27]:
import easydict as parser
 
opt = parser.EasyDict({
        "n_epochs": 50,
        "batch_size": 64, ### default: 64
        "lr": 0.0002,
        "b1": 0.5,
        "b2": 0.999,
        "n_cpu": 8,
        "latent_dim": 100,
        "label_dim": 10, ### label dimension
        "n_classes": 10,
        "img_size": 32,
        "channels": 1,
        "sample_interval": 400
})

img_shape = (opt.channels, opt.img_size, opt.img_size)
print(">>> Image shape:", img_shape)
cuda = True if torch.cuda.is_available() else False
print(">>> GPU Usage:", cuda)

>>> Image shape: (1, 32, 32)
>>> GPU Usage: True


## Dataloader

In [28]:
# Configure data loader
os.makedirs("data/mnist", exist_ok=True)

# Dataloader
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    drop_last=True, ### ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 256])
    shuffle=True,
)

## Saving sample image func

In [29]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    labels = F.one_hot(labels, num_classes=10) ### One hot encoding
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

## Generator Class

In [30]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        #self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes) ### One hot encoding

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        #gen_input = torch.cat((self.label_emb(labels), noise), -1)
        gen_input = torch.cat((labels, noise), -1) ### One hot encoding

        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img

## Discriminator Class

In [31]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        #self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes) ### One hot encoding

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        #d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        d_in = torch.cat((img.view(img.size(0), -1), labels), -1) ### One hot encoding
        validity = self.model(d_in)
        
        return validity

## Loss function and Initializing Model

In [32]:
# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

# Loss functions
adversarial_loss = torch.nn.MSELoss()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

## Optimizers

In [33]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

## Training

In [34]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))
        labels = F.one_hot(labels, num_classes=10) ### One hot encoding

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size))) ### ?????
        gen_labels = F.one_hot(gen_labels, num_classes=10) ### One hot encoding
        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Epoch 44/50] [Batch 622/937] [D loss: 0.196091] [G loss: 0.650649]
[Epoch 44/50] [Batch 623/937] [D loss: 0.182746] [G loss: 0.415008]
[Epoch 44/50] [Batch 624/937] [D loss: 0.179837] [G loss: 0.362689]
[Epoch 44/50] [Batch 625/937] [D loss: 0.178969] [G loss: 0.386782]
[Epoch 44/50] [Batch 626/937] [D loss: 0.174132] [G loss: 0.539943]
[Epoch 44/50] [Batch 627/937] [D loss: 0.209901] [G loss: 0.414862]
[Epoch 44/50] [Batch 628/937] [D loss: 0.167446] [G loss: 0.439880]
[Epoch 44/50] [Batch 629/937] [D loss: 0.182581] [G loss: 0.442359]
[Epoch 44/50] [Batch 630/937] [D loss: 0.154819] [G loss: 0.446390]
[Epoch 44/50] [Batch 631/937] [D loss: 0.172949] [G loss: 0.487191]
[Epoch 44/50] [Batch 632/937] [D loss: 0.161535] [G loss: 0.387424]
[Epoch 44/50] [Batch 633/937] [D loss: 0.177964] [G loss: 0.625014]
[Epoch 44/50] [Batch 634/937] [D loss: 0.180520] [G loss: 0.264440]
[Epoch 44/50] [Batch 635/937] [D loss: 0.159823] [G loss: 0.531262]
[Epoch 44/50

## Save Model

In [35]:
torch.save(generator.state_dict(), 'generator.pth')

## Loading trained Model

In [36]:
trained_generator = Generator().to('cuda')
trained_generator.load_state_dict(torch.load('generator.pth'))
trained_generator.eval()

Generator(
  (model): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Linear(in_features=1024, out_features=1024, bias=True)
    (12): Tanh()
  )
)

## Saving label image

In [68]:
label = [# zero
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],

         # three
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],

         # seven
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],

         # zero + three
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],

         # zero + seven
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         ] 

In [75]:
# noise z
z = Variable(FloatTensor(np.random.normal(0, 1, (len(label), opt.latent_dim))))

# labels
labels = np.array(label)
labels = Variable(LongTensor(labels))

# generating image
gen_img = trained_generator(z, labels)

# image saving
save_image(gen_img.data, "generated_image.jpg", normalize=True)